In [ ]:
!pip install easyfsl

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
from tqdm import tqdm
import numpy as np
import time

image_size = 28


train_set = Omniglot(
    root="./data",
    background=True,
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]
    ),
    download=True,
)
test_set = Omniglot(
    root="./data",
    background=False,
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize([int(image_size * 1.15), int(image_size * 1.15)]),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    ),
    download=True,
)
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=10), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=7), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=4), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4), nn.ReLU()
        )
        self.fc1 = nn.Sequential(nn.Linear(256*6*6, 4096), nn.Sigmoid())
        self.fc2 = nn.Sequential(nn.Linear(4096, 1), nn.Sigmoid())  # Add sigmoid activation function

    def forward(self, x1, x2):
        out1 = self.conv1(x1)
        out1 = self.conv2(out1)
        out1 = self.conv3(out1)
        out1 = self.conv4(out1)
        out1 = out1.view(out1.size()[0], -1)
        out1 = self.fc1(out1)

        out2 = self.conv1(x2)
        out2 = self.conv2(out2)
        out2 = self.conv3(out2)
        out2 = self.conv4(out2)
        out2 = out2.view(out2.size()[0], -1)
        out2 = self.fc1(out2)

        # Euclidean distance
        distance = torch.abs(out1 - out2)
        out = self.fc2(distance)
        return out






Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def train_siamese_network(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (img1, img2, label) in enumerate(train_loader):
            img1, img2, label = img1, img2, label

            optimizer.zero_grad()
            output = model(img1, img2)
            loss = criterion(output.squeeze(), label)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")


In [ ]:
def test_siamese_network(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for img1, img2, label in test_loader:
            img1, img2, label = img1, img2, label
            output = model(img1, img2)
            predicted = torch.round(output)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    accuracy = correct / total
    return accuracy

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Example layers, adjust as necessary
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(16 * 103 * 103, 1)  # Adjust based on output size

    def forward(self, img1, img2):
        # Implement forward pass logic
        x1 = self.conv1(img1)
        x2 = self.conv1(img2)
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        output = self.fc1(x1) - self.fc1(x2)  # Example output logic
        return output

def calculate_gflops(model, input_size, batch_size=1):
    print("Starting GFLOPS calculation...")
    flops = 0
    input_tensor1 = torch.randn(batch_size, *input_size).to(next(model.parameters()).device)
    input_tensor2 = torch.randn(batch_size, *input_size).to(next(model.parameters()).device)

    # Hook functions
    def conv2d_hook(module, input, output):
        # Retrieve batch size from the input shape
        batch_size = input[0].shape[0]
        out_channels = output.shape[1]
        out_h, out_w = output.shape[2:4]
        kernel_h, kernel_w = module.kernel_size
        in_channels = module.in_channels
        flops = batch_size * out_channels * out_h * out_w * (kernel_h * kernel_w * in_channels)
        module.__flops__ = flops
        print(f"Conv2d Layer FLOPs: {flops}")

    def linear_hook(module, input, output):
        # Retrieve batch size from the input shape
        batch_size = input[0].shape[0]
        flops = batch_size * module.in_features * module.out_features
        module.__flops__ = flops
        print(f"Linear Layer FLOPs: {flops}")

    # Attach hooks
    hooks = []
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            hooks.append(module.register_forward_hook(conv2d_hook))
        elif isinstance(module, nn.Linear):
            hooks.append(module.register_forward_hook(linear_hook))

    # Forward pass to trigger hooks
    print("Performing forward pass...")
    model(input_tensor1, input_tensor2)

    # Sum up the FLOPs
    for module in model.modules():
        if hasattr(module, '__flops__'):
            flops += module.__flops__

    # Remove hooks after use
    for hook in hooks:
        hook.remove()

    # Convert FLOPs to GFLOPS
    gflops = flops / 1e9
    print(f"Total FLOPs: {flops}")
    return gflops

# Example Usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
input_size = (1, 105, 105)  # Input shape
gflops = calculate_gflops(model, input_size, batch_size=1)
print(f"GFLOPS: {gflops:.4f}")


Starting GFLOPS calculation...
Performing forward pass...
Conv2d Layer FLOPs: 1527696
Conv2d Layer FLOPs: 1527696
Linear Layer FLOPs: 169744
Linear Layer FLOPs: 169744
Total FLOPs: 1697440
GFLOPS: 0.0017


In [ ]:
def calculate_model_size(model):
    # Total parameters count
    model_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    model_size_mb = model_parameters * 4 / (1024 ** 2)  # 4 bytes per parameter (float32)
    print(f"Model size: {model_size_mb:.2f} MB")
    return model_size_mb

# Example usage
model_size = calculate_model_size(model)


Model size: 0.65 MB


In [ ]:
import time

def calculate_inference_time(model, input_size, device, num_iterations=100):
    model.eval()
    inputs1 = torch.randn(1, *input_size).to(device)
    inputs2 = torch.randn(1, *input_size).to(device)

    start_time = time.time()
    with torch.no_grad():
        for _ in range(num_iterations):
            _ = model(inputs1, inputs2)

    total_time = time.time() - start_time
    avg_inference_time = total_time / num_iterations
    print(f"Average inference time per image pair: {avg_inference_time:.6f} seconds")
    return avg_inference_time

# Example usage
avg_time = calculate_inference_time(model, input_size, device)


Average inference time per image pair: 0.000281 seconds


In [ ]:
def calculate_accuracy(model, dataloader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data in dataloader:
            # Adjust the unpacking based on your dataset
            if len(data) == 3:
                inputs1, inputs2, labels = data
            elif len(data) == 2:
                inputs1, labels = data
                # If you're using a single input for comparison, you'll need to adjust how you handle this
                inputs2 = inputs1  # Example: using the same input for the sake of demonstration

            inputs1, inputs2, labels = inputs1.to(device), inputs2.to(device), labels.to(device)

            outputs = model(inputs1, inputs2)
            predicted = (outputs > 0.5).float()  # Threshold for binary classification
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.4f}")
    return accuracy

# Example usage (ensure validation_loader is defined and populated)
# accuracy = calculate_accuracy(model, validation_loader, device)


In [ ]:
# Assuming you are using a custom dataset
train_dataset = Omniglot(root='data', background=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create a validation dataset and loader (you can split the original dataset)
validation_dataset = Omniglot(root='data', background=False, download=True, transform=transform)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Now you can use this validation_loader in the accuracy function
accuracy = calculate_accuracy(model, validation_loader, device)


Files already downloaded and verified
Files already downloaded and verified
Accuracy: 0.0486


In [ ]:
# Assuming you have defined the training DataLoader somewhere
# We'll split the Omniglot dataset into training and validation sets

from torchvision.datasets import Omniglot
from torch.utils.data import random_split

# Define the transformation (same as training)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((105, 105)),
    transforms.ToTensor()
])

# Load the Omniglot dataset
full_dataset = Omniglot(root='./data', transform=transform, download=True)

# Split the dataset into training and validation (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Now you can use `validation_loader` in your accuracy function


Files already downloaded and verified
